# Holiday Effect Strategy - Options Strategy

Implement put-selling overlay strategy around holiday events.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from options_strategy import OptionsStrategy
from data_acquisition import DataAcquisition
from signal_generator import SignalGenerator

plt.style.use('seaborn-v0_8-darkgrid')

## 1. Load Data and Generate Event Windows

In [ ]:
# Load data
data_acq = DataAcquisition('config.yaml')
dataset = data_acq.fetch_full_dataset()

# Generate event windows
signal_gen = SignalGenerator('config.yaml')
signals, windows = signal_gen.generate_signal_series(dataset['amzn_prices'].index)

print(f"Total event windows: {len(windows)}")
print(f"Black Friday events: {len(windows[windows['event_type'] == 'black_friday'])}")
print(f"Prime Day events: {len(windows[windows['event_type'] == 'prime_day'])}")

# Use only 2012+ per research
recent_windows = windows[windows['year'] >= 2012]
print(f"\nRecent windows (2012+): {len(recent_windows)}")

## 2. Simulate Put Selling Strategy

In [ ]:
# Initialize options strategy
options_strat = OptionsStrategy('config.yaml')

# Simulate trades
trades = options_strat.simulate_options_trades(
    recent_windows,
    dataset['amzn_prices'],
    initial_capital=1000000
)

print("=== Options Trades ===")
print(trades[['entry_date', 'event_type', 'entry_price', 'exit_price', 
              'strike', 'premium_collected', 'pnl', 'outcome']].head(10))

## 3. Performance Metrics

In [ ]:
# Calculate metrics
metrics = options_strat.calculate_metrics(trades)

print("=== Options Strategy Performance ===")
print(f"Total Trades:          {metrics['total_trades']}")
print(f"Winning Trades:        {metrics['winning_trades']}")
print(f"Win Rate:              {metrics['win_rate']*100:.1f}%")
print(f"Total Premium:         ${metrics['total_premium_collected']:,.2f}")
print(f"Total PnL:             ${metrics['total_pnl']:,.2f}")
print(f"Avg Premium/Trade:     ${metrics['avg_premium_per_trade']:,.2f}")
print(f"Avg PnL/Trade:         ${metrics['avg_pnl_per_trade']:,.2f}")
print(f"Final Portfolio Value: ${metrics['final_portfolio_value']:,.2f}")

# Calculate return
total_return = (metrics['final_portfolio_value'] - 1000000) / 1000000 * 100
print(f"Total Return:          {total_return:.2f}%")

## 4. Trade Outcomes Visualization

In [ ]:
# Win/Loss distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Outcome counts
outcome_counts = trades['outcome'].value_counts()
axes[0].pie(outcome_counts, labels=outcome_counts.index, autopct='%1.1f%%',
            colors=['green', 'red'], startangle=90)
axes[0].set_title('Trade Outcomes')

# PnL distribution
axes[1].hist(trades['pnl'], bins=20, edgecolor='black', alpha=0.7)
axes[1].axvline(0, color='red', linestyle='--', linewidth=2)
axes[1].set_title('Trade PnL Distribution')
axes[1].set_xlabel('PnL ($)')
axes[1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

## 5. Cumulative Performance

In [ ]:
# Plot cumulative portfolio value
fig, ax = plt.subplots(figsize=(14, 7))
ax.plot(trades['exit_date'], trades['portfolio_value'], linewidth=2, marker='o')
ax.axhline(1000000, color='red', linestyle='--', label='Initial Capital')
ax.set_title('Options Strategy: Cumulative Portfolio Value', fontsize=14, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Portfolio Value ($)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Event Type Analysis

In [ ]:
# Compare performance by event type
event_performance = trades.groupby('event_type').agg({
    'pnl': ['mean', 'sum', 'count'],
    'premium_collected': 'mean',
    'outcome': lambda x: (x == 'win').sum() / len(x)
})

print("=== Performance by Event Type ===")
print(event_performance)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Average PnL by event
event_pnl = trades.groupby('event_type')['pnl'].mean()
event_pnl.plot(kind='bar', ax=axes[0], color=['blue', 'orange'])
axes[0].set_title('Average PnL by Event Type')
axes[0].set_ylabel('Average PnL ($)')
axes[0].set_xlabel('Event Type')
axes[0].axhline(0, color='red', linestyle='--')

# Win rate by event
event_winrate = trades.groupby('event_type')['outcome'].apply(lambda x: (x == 'win').sum() / len(x) * 100)
event_winrate.plot(kind='bar', ax=axes[1], color=['blue', 'orange'])
axes[1].set_title('Win Rate by Event Type')
axes[1].set_ylabel('Win Rate (%)')
axes[1].set_xlabel('Event Type')
axes[1].set_ylim([0, 100])

plt.tight_layout()
plt.show()

## 7. Risk Analysis

In [ ]:
# Calculate risk metrics
returns = trades['pnl'] / 1000000  # As fraction of initial capital

print("=== Risk Metrics ===")
print(f"Average Return per Trade: {returns.mean()*100:.4f}%")
print(f"Std Dev of Returns:       {returns.std()*100:.4f}%")
print(f"Sharpe Ratio (approx):    {returns.mean() / returns.std() if returns.std() > 0 else 0:.2f}")
print(f"Max Loss:                 ${trades['pnl'].min():,.2f}")
print(f"Max Gain:                 ${trades['pnl'].max():,.2f}")
print(f"Profit Factor:            {trades[trades['pnl'] > 0]['pnl'].sum() / abs(trades[trades['pnl'] < 0]['pnl'].sum()) if len(trades[trades['pnl'] < 0]) > 0 else float('inf'):.2f}")

## 8. Combined Strategy (Equity + Options)

In [ ]:
# Simulate combined portfolio
# 95% SPY base + 5% options overlay

spy_allocation = 0.95
options_allocation = 0.05

# SPY buy-and-hold returns
spy_recent = dataset['spy_prices'].loc[recent_windows.iloc[0]['entry_date']:]
spy_returns = (spy_recent['Adj Close'].iloc[-1] - spy_recent['Adj Close'].iloc[0]) / spy_recent['Adj Close'].iloc[0]

# Options returns
options_returns = (metrics['final_portfolio_value'] - 1000000) / 1000000

# Combined returns
combined_return = spy_allocation * spy_returns + options_allocation * options_returns

print("=== Combined Strategy ===")
print(f"SPY Component (95%):      {spy_returns*100:.2f}%")
print(f"Options Component (5%):   {options_returns*100:.2f}%")
print(f"Combined Return:          {combined_return*100:.2f}%")
print(f"\nEnhancement from Options: {(combined_return - spy_returns)*100:.2f}%")